In [ ]:
!pip install tensorflow==1.14.0
import tensorflow as tf
tf.__version__

In [2]:
import pandas as pd
from matplotlib import pyplot as plt
import pandas as pd
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.stattools import adfuller
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import pearsonr
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation, Dropout, Flatten
from keras.optimizers import Adam
import shap
import warnings
warnings.simplefilter("ignore")

Using TensorFlow backend.


In [3]:
data_filename = 'datasets/owid-covid-data.csv'
covid_df = pd.read_csv(data_filename)
covid_df['timestamp'] = pd.to_datetime(covid_df['date'], format="%Y-%m-%d")
covid_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 343286 entries, 0 to 343285
Data columns (total 68 columns):
iso_code                                      343286 non-null object
continent                                     326978 non-null object
location                                      343286 non-null object
date                                          343286 non-null object
total_cases                                   305500 non-null float64
new_cases                                     333872 non-null float64
new_cases_smoothed                            332613 non-null float64
total_deaths                                  284073 non-null float64
new_deaths                                    333905 non-null float64
new_deaths_smoothed                           332675 non-null float64
total_cases_per_million                       305500 non-null float64
new_cases_per_million                         333872 non-null float64
new_cases_smoothed_per_million                332613 

In [4]:
# 1. Filter the DataFrame by Date Range
start_date, end_date = "2020-01-03", "2022-12-31"
covid_df = covid_df[(covid_df['date'] >= start_date) & (covid_df['date'] <= end_date)]

# 2. Count Samples and Features
print(f"The total number of samples is {len(covid_df)}, with each sample corresponding to {len(covid_df.columns)} features.")

# 3. Count Missing Values
miss_vals = covid_df.isnull().sum().sort_values(ascending=False)
pd.DataFrame([miss_vals])

The total number of samples is 276734, with each sample corresponding to 68 features.


weekly_icu_admissions  weekly_icu_admissions_per_million  excess_mortality  \
0                 268463                             268463            266489   

   excess_mortality_cumulative  excess_mortality_cumulative_absolute  \
0                       266489                                266489   

   excess_mortality_cumulative_per_million  weekly_hosp_admissions  \
0                                   266489                  257034   

   weekly_hosp_admissions_per_million  icu_patients  icu_patients_per_million  \
0                              257034        243890                    243890   

   ...  new_deaths_smoothed_per_million  new_cases_per_million  new_cases  \
0  ...                             8863                   7661       7661   

   new_deaths_per_million  new_deaths  location  timestamp  date  population  \
0                    7633        7633         0          0     0           0   

   iso_code  
0         0  

[1 rows x 68 columns]

In [5]:
# 4. Handle Missing Values
covid_df['continent'].fillna('OWID', inplace=True)
covid_df['tests_units'].fillna('no info', inplace=True)
covid_df.fillna(0, inplace=True)

# 5. Check for Remaining Missing Values
if covid_df.isnull().sum().any():
    print("There are still missing values in the DataFrame.")
else:
    print("All missing values have been taken care of.")

All missing values have been taken care of.


In [6]:
covid_df = covid_df.sort_values(by='date')

In [7]:
feature_columns = ["icu_patients_per_million", "reproduction_rate", "hosp_patients_per_million", "new_people_vaccinated_smoothed_per_hundred", "new_vaccinations_smoothed_per_million", "stringency_index", "tests_per_case"]
forecast_column = ["new_cases_smoothed_per_million"]

In [8]:
# Selecting relevant features
covid_df = covid_df[['iso_code'] + ['date'] + feature_columns + forecast_column]

# Setting the date as the index
covid_df.set_index('date', inplace=True)

# Explainable VAR Forecasting 

In [9]:
def prepare_data(df, iso_code):
    iso_df = df[df['iso_code'] == iso_code]
    iso_df = iso_df.drop('iso_code', axis=1)
    iso_df = check_stationarity_and_difference(iso_df)
    return iso_df

def check_stationarity_and_difference(dataframe):
    for column in dataframe.columns:
        # Check if the series is constant
        if dataframe[column].nunique() <= 1:
            print(f"Column {column} is constant. Skipping ADF test.")
            dataframe = dataframe.drop(column, axis = 1)
            continue

        # Perform the ADF test
        result = adfuller(dataframe[column])
        if result[1] > 0.05:  # p-value is greater than 0.05
            # Apply differencing
            dataframe[column] = dataframe[column].diff().dropna()

    return dataframe.dropna()

def split_and_scale_data(dataframe, train_ratio):
    split_idx = int(len(iso_covid_df) * 0.9)
    train_data, test_data = iso_covid_df[:split_idx], iso_covid_df[split_idx:]
    scaler = MinMaxScaler()
    train_scaled = scaler.fit_transform(train_data)
    test_scaled = scaler.transform(test_data)
    return train_scaled, test_scaled, scaler, train_data, test_data

def fit_var_model(train_scaled, train_data, maxlags=15):
    model = VAR(train_scaled, dates=train_data.index)
    model_fitted = model.fit(maxlags=15, ic='aic')  # Using Akaike Information Criterion to find optimal lag
    return model_fitted

def forecast_and_evaluate(model_fitted, train_scaled, test_data, scaler, forecast_column):
    forecast_steps = len(test_data)
    forecast = model_fitted.forecast(train_scaled[-model_fitted.k_ar:], steps=forecast_steps)
    # Inverse transform the forecast
    forecast = scaler.inverse_transform(forecast)
    # Inverse transform the test data (for evaluation purposes)
    test_data_inverse = scaler.inverse_transform(test_scaled)
    forecast_df = pd.DataFrame(forecast, columns=test_data.columns, index=test_data.index)
    mae, rmse = evaluate_forecast(test_data_inverse[:, -1], forecast_df[forecast_column[0]].values)
    print(f"\nMAE: {mae}, RMSE: {rmse}")
    return forecast

def evaluate_forecast(actual, predicted):
    mae = mean_absolute_error(actual, predicted)
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    return mae, rmse

def calculate_correlations(dataframe, feature_columns, forecast_column):
    correlations = {}
    for feature in feature_columns:
        corr, _ = pearsonr(dataframe[feature], dataframe[forecast_column[0]])
        correlations[feature] = corr
    sorted_features = sorted(correlations.items(), key=lambda x: abs(x[1]), reverse=True)
    return sorted_features

def plot_predictions(original_df, combined_predictions_df, forecast_column):
    plt.figure(figsize=(12, 6))
    plt.plot(original_df[forecast_column[0]], label='Original', color='blue')
    plt.plot(combined_predictions_df[forecast_column[0]], label='Predicted', color='red')
    plt.title('Fitted Model Predictions vs Original Data')
    plt.xlabel('Date')
    plt.ylabel(forecast_column[0])
    plt.legend()
    plt.show()

## Testing MAE, RMSE Metrics + 
## Global Explainability Analysis of features (w.r.t. Pearson Correlation coefficients) 
## for all 4 countries - "JPN" (Japan), "ITA" (Italy), "USA" (United States of America), "FRA" (France)

In [34]:
iso_codes = ["JPN", "ITA", "USA", "FRA"]
for iso_code in iso_codes:
    print(f"\nProcessing for country: {iso_code}")

    # Data Preparation
    iso_covid_df = prepare_data(covid_df, iso_code)

    # Split and Scale Data
    train_scaled, test_scaled, scaler, train_data, test_data = split_and_scale_data(iso_covid_df, train_ratio=0.8)

    # Fit VAR Model
    model_fitted = fit_var_model(train_scaled, train_data)

    # Forecast and Evaluate
    forecast = forecast_and_evaluate(model_fitted, train_scaled, test_data, scaler, forecast_column)

    # Generate Combined Predictions
    train_predictions = model_fitted.fittedvalues
    train_predictions = scaler.inverse_transform(train_predictions)
    combined_predictions = np.concatenate((train_predictions, forecast), axis=0)
    combined_predictions_df = pd.DataFrame(combined_predictions, columns=train_data.columns, index=iso_covid_df.index[model_fitted.k_ar:len(iso_covid_df)])

    # Correlation Analysis
    sorted_features = calculate_correlations(combined_predictions_df, list(set(feature_columns) & set(combined_predictions_df.columns)), forecast_column)
    print(f"\nFeature Importance in forecasting new case count (based on Pearson Correlation) for country: {iso_code}:\n")
    for feature, corr in sorted_features:
        print(f"{feature}: {corr}")
    print()


Processing for country: JPN

MAE: 448.48740855017974, RMSE: 633.4510836951887

Feature Importance in forecasting new case count (based on Pearson Correlation) for country: JPN:

new_vaccinations_smoothed_per_million: 0.24504706883921917
tests_per_case: -0.22562999798424832
hosp_patients_per_million: 0.18070253174309941
icu_patients_per_million: 0.10772914138159594
stringency_index: -0.028809746609959917
reproduction_rate: 0.007736543072367508
new_people_vaccinated_smoothed_per_hundred: 0.0018065816210661937


Processing for country: ITA

MAE: 88.43733173994447, RMSE: 114.7199314987372

Feature Importance in forecasting new case count (based on Pearson Correlation) for country: ITA:

tests_per_case: -0.3942610559890737
hosp_patients_per_million: 0.24707108626866256
new_vaccinations_smoothed_per_million: -0.1137718510763623
stringency_index: -0.0942048930629389
icu_patients_per_million: 0.07675727069492683
reproduction_rate: 0.017490893752928456
new_people_vaccinated_smoothed_per_hundre

# Explainable LSTM Forecasting

In [11]:
def create_dataset(X, y, time_steps):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

def split_and_scale_data_lstm(dataframe, train_ratio):

    split_idx = int(len(dataframe) * 0.9)
    train_data, test_data = dataframe[:split_idx], dataframe[split_idx:]
    time_steps = 20  # use 20 past days to predict the next day
    train_X, train_y = create_dataset(train_data.iloc[:, :-1], train_data.iloc[:, -1], time_steps)
    test_X, test_y = create_dataset(test_data.iloc[:, :-1], test_data.iloc[:, -1], time_steps)

    # Scaling the data
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_X_scaled = scaler.fit_transform(train_X.reshape(-1, train_X.shape[2])).reshape(train_X.shape)
    test_X_scaled = scaler.transform(test_X.reshape(-1, test_X.shape[2])).reshape(test_X.shape)
    return train_X_scaled, train_y, test_X_scaled, test_y, scaler

def inverse_transform_prediction(yhat, scaler, n_features):
    dummy = np.zeros((len(yhat), n_features))
    dummy[:, 0] = yhat.ravel()
    return scaler.inverse_transform(dummy)[:, 0]

def createLSTMModel(inputShape, l1Nodes=64, d1Nodes=64, d2Nodes=32):
    lstm1 = LSTM(l1Nodes, input_shape=inputShape)
    # lstm2 = LSTM(l2Nodes)
    # flatten = Flatten()
    dense1 = Dense(d1Nodes)
    dense2 = Dense(d2Nodes)
    outL = Dense(1)
    # combine the layers
    layers = [lstm1, dense1, dense2, outL]
    # create the model
    model = Sequential(layers)
    model.compile(loss='mae', optimizer=Adam(learning_rate=0.007))
    return model

def evaluate_forecast(actual, predicted):
    mae = mean_absolute_error(actual, predicted)
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    return mae, rmse

def calculate_correlations(dataframe, feature_columns, forecast_column):
    correlations = {}
    for feature in feature_columns:
        corr, _ = pearsonr(dataframe[feature], dataframe[forecast_column[0]])
        correlations[feature] = corr
    sorted_features = sorted(correlations.items(), key=lambda x: abs(x[1]), reverse=True)
    return sorted_features

    # # input layer
    # lstm1 = LSTM(l1Nodes, input_shape=inputShape)
    # # lstm2 = LSTM(l2Nodes)
    # # flatten = Flatten()
    # dense1 = Dense(d1Nodes)
    # dense2 = Dense(d2Nodes)
    # outL = Dense(1)
    # # combine the layers
    # layers = [lstm1, dense1, dense2, outL]
    # # create the model
    # model = Sequential(layers)
    # opt = Adam(learning_rate=0.005)
    # model.compile(optimizer="adam", loss='rmse')
    # return model

## Training and Testing LSTM Model (w.r.t. MAE, RMSE) for "JPN" (Japan)

In [112]:
iso_code = "JPN"
print(f"\nProcessing for country: {iso_code}")

# Data Preparation
iso_covid_df = prepare_data(covid_df, iso_code)

# Split and Scale Data
train_X_scaled, train_y, test_X_scaled, test_y, scaler = split_and_scale_data_lstm(iso_covid_df, train_ratio=0.8)

# Fit LSTM Model
# Define LSTM model
model = createLSTMModel((train_X_scaled.shape[1], train_X_scaled.shape[2]), l1Nodes=128, d1Nodes=64, d2Nodes=32)

# Fit model
history = model.fit(train_X_scaled, train_y, epochs=100, batch_size=8, validation_data=(test_X_scaled, test_y), verbose=0, shuffle=False)
print("\nLSTM Model Trained!\n")
yhat = model.predict(test_X_scaled)

# Evaluate the model
mae, rmse = evaluate_forecast(test_y, yhat)
print("\nTesting (Out-of-Sample) Data Performance Metrics:")
print(f"\nMAE: {mae}, RMSE: {rmse}")


Processing for country: JPN

LSTM Model Trained!


Testing (Out-of-Sample) Data Performance Metrics:

MAE: 596.256809540134, RMSE: 691.4264635466703


## Global Explainability Analysis of features (w.r.t. Pearson Correlation coefficients) for "JPN" (Japan)

In [113]:
train_yhat = model.predict(train_X_scaled)# Concatenate the training and testing predictions and actual values
all_predictions = np.concatenate((train_yhat.ravel(), yhat.ravel()))
all_actual = np.concatenate((train_y, test_y))

# Now, create a DataFrame that aligns the predictions with the true values
predictions_df = pd.DataFrame({
    'Predicted': all_predictions,
    'Actual': all_actual
})

# Calculate correlations for each feature in the original dataset against the actual target
correlations = {}
for feature in feature_columns:
    # Extract the feature data from the original DataFrame, making sure to align it with the concatenated predictions
    # This assumes that the feature data is properly aligned with the target variable
    feature_data_aligned = iso_covid_df[feature][-len(all_predictions):].values
    corr, _ = pearsonr(feature_data_aligned, predictions_df['Actual'])
    correlations[feature] = corr

# Sort features by their correlation strength
sorted_features = sorted(correlations.items(), key=lambda x: abs(x[1]), reverse=True)

# Output the sorted features and their correlations
print(f"\nFeature Importance in forecasting new case count (based on Pearson Correlation) for country: {iso_code}:\n")
for feature, corr in sorted_features:
    print(f"{feature}: {corr}")


Feature Importance in forecasting new case count (based on Pearson Correlation) for country: JPN:

new_vaccinations_smoothed_per_million: 0.2500258316822231
tests_per_case: -0.2462978857360657
stringency_index: -0.22503389342819877
hosp_patients_per_million: 0.20475544926690667
reproduction_rate: -0.1540072972898024
icu_patients_per_million: 0.09516849842360542
new_people_vaccinated_smoothed_per_hundred: -0.0034638044425013984


## Local / Phased Explainability Analysis of features (w.r.t. SHAP values) for "JPN" (Japan)

In [126]:
# Use the training data for deep explainer => can use fewer instances
explainer = shap.DeepExplainer(model, train_X_scaled)
# explain the the testing instances (can use fewer instanaces)
# explaining each prediction requires 2 * background dataset size runs
shap_values = explainer.shap_values(train_X_scaled[915:918])
# init the JS visualization code
shap.initjs()
print("SHAP Contribution to Prediction at date 2022-07-29 - period of fastest growth for JPN (Omicron variant onset):\n")
shap.force_plot(explainer.expected_value[0], shap_values[0][2], train_X_scaled[917], feature_names = feature_columns, out_names=forecast_column)

SHAP Contribution to Prediction at date 2022-07-29 - period of fastest growth for JPN (Omicron variant onset):



## Training and Testing LSTM Model for "USA" (United States of America)

In [70]:
iso_code = "USA"
print(f"\nProcessing for country: {iso_code}")

# Data Preparation
iso_covid_df = prepare_data(covid_df, iso_code)

# Split and Scale Data
train_X_scaled, train_y, test_X_scaled, test_y, scaler = split_and_scale_data_lstm(iso_covid_df, train_ratio=0.8)

# Fit LSTM Model
# Define LSTM model
model = createLSTMModel((train_X_scaled.shape[1], train_X_scaled.shape[2]), l1Nodes=64, d1Nodes=32, d2Nodes=16)

# Fit model
history = model.fit(train_X_scaled, train_y, epochs=100, batch_size=16, validation_data=(test_X_scaled, test_y), verbose=0, shuffle=False)
print("\nLSTM Model Trained!\n")
yhat = model.predict(test_X_scaled)

# Evaluate the model
mae, rmse = evaluate_forecast(test_y, yhat)
print("\nTesting (Out-of-Sample) Data Performance Metrics:")
print(f"\nMAE: {mae}, RMSE: {rmse}")


Processing for country: USA

LSTM Model Trained!


Testing (Out-of-Sample) Data Performance Metrics:

MAE: 163.93053755425348, RMSE: 165.18676166054772


## Global Explainability Analysis of features (w.r.t. Pearson Correlation coefficients) for "USA" (United States of America)

In [71]:
train_yhat = model.predict(train_X_scaled)# Concatenate the training and testing predictions and actual values
all_predictions = np.concatenate((train_yhat.ravel(), yhat.ravel()))
all_actual = np.concatenate((train_y, test_y))

# Now, create a DataFrame that aligns the predictions with the true values
predictions_df = pd.DataFrame({
    'Predicted': all_predictions,
    'Actual': all_actual
})

# Calculate correlations for each feature in the original dataset against the actual target
correlations = {}
for feature in feature_columns:
    # Extract the feature data from the original DataFrame, making sure to align it with the concatenated predictions
    # This assumes that the feature data is properly aligned with the target variable
    feature_data_aligned = iso_covid_df[feature][-len(all_predictions):].values
    corr, _ = pearsonr(feature_data_aligned, predictions_df['Actual'])
    correlations[feature] = corr

# Sort features by their correlation strength
sorted_features = sorted(correlations.items(), key=lambda x: abs(x[1]), reverse=True)

# Output the sorted features and their correlations
print(f"\nFeature Importance in forecasting new case count (based on Pearson Correlation) for country: {iso_code}:\n")
for feature, corr in sorted_features:
    print(f"{feature}: {corr}")


Feature Importance in forecasting new case count (based on Pearson Correlation) for country: USA:

hosp_patients_per_million: 0.6026853274191072
icu_patients_per_million: 0.5064149199591723
reproduction_rate: -0.2822523443262936
tests_per_case: -0.11627009218044355
stringency_index: -0.058846594019735195
new_vaccinations_smoothed_per_million: -0.020631853883341048
new_people_vaccinated_smoothed_per_hundred: 0.016062759600049298


## Local / Phased Explainability Analysis of features (w.r.t. SHAP values) for "USA" (United States of America)

In [73]:
# Use the training data for deep explainer => can use fewer instances
explainer = shap.DeepExplainer(model, train_X_scaled)
# explain the the testing instances (can use fewer instanaces)
# explaining each prediction requires 2 * background dataset size runs
shap_values = explainer.shap_values(train_X_scaled[708:711,:,:])
# init the JS visualization code
shap.initjs()
print("SHAP Contribution to Prediction at date 2022-01-03 - period of fastest growth for USA (Omicron variant onset):\n")
shap.force_plot(explainer.expected_value[0], shap_values[0][2], train_X_scaled[710], feature_names = feature_columns, out_names=forecast_column)

SHAP Contribution to Prediction at date 2022-01-03 - period of fastest growth for USA (Omicron variant onset):



## (ONLY for USA) Further Local / Phased Explainability Analysis of SHAP feature contribution across 3 historically prominent phases of pandemic

In [83]:
# Use the training data for deep explainer => can use fewer instances
explainer = shap.DeepExplainer(model, train_X_scaled)
# explain the the testing instances (can use fewer instanaces)
# explaining each prediction requires 2 * background dataset size runs
shap_values = explainer.shap_values(train_X_scaled[312:314,:,:])
# init the JS visualization code
shap.initjs()
print("SHAP Contribution to Prediction at date 2020-12-02 - period of fastest growth in 2020 (before vaccines were distributed and Omicron) for USA:\n")
shap.force_plot(explainer.expected_value[0], shap_values[0][1], train_X_scaled[313], feature_names = feature_columns, out_names=forecast_column)

SHAP Contribution to Prediction at date 2020-12-02 - period of fastest growth in 2020 (before vaccines were distributed and Omicron) for USA:



In [85]:
# Use the training data for deep explainer => can use fewer instances
explainer = shap.DeepExplainer(model, train_X_scaled)
# explain the the testing instances (can use fewer instanaces)
# explaining each prediction requires 2 * background dataset size runs
shap_values = explainer.shap_values(train_X_scaled[769:771,:,:])
# init the JS visualization code
shap.initjs()
print("SHAP Contribution to Prediction at date 2022-03-04 - period of fastest decline (after vaccines were distributed and Omicron spread almost everywhere) for USA:\n")
shap.force_plot(explainer.expected_value[0], shap_values[0][1], train_X_scaled[770], feature_names = feature_columns, out_names=forecast_column)

SHAP Contribution to Prediction at date 2022-03-04 - period of fastest decline (after vaccines were distributed and Omicron spread almost everywhere) for USA:



## Training and Testing LSTM Model for "FRA" (France)

In [88]:
iso_code = "FRA"
print(f"\nProcessing for country: {iso_code}")

# Data Preparation
iso_covid_df = prepare_data(covid_df, iso_code)

# Split and Scale Data
train_X_scaled, train_y, test_X_scaled, test_y, scaler = split_and_scale_data_lstm(iso_covid_df, train_ratio=0.8)

# Fit LSTM Model
# Define LSTM model
model = createLSTMModel((train_X_scaled.shape[1], train_X_scaled.shape[2]), l1Nodes=128, d1Nodes=64, d2Nodes=32)

# Fit model
history = model.fit(train_X_scaled, train_y, epochs=100, batch_size=8, validation_data=(test_X_scaled, test_y), verbose=0, shuffle=False)
print("\nLSTM Model Trained!\n")
yhat = model.predict(test_X_scaled)

# Evaluate the model
mae, rmse = evaluate_forecast(test_y, yhat)
print("\nTesting (Out-of-Sample) Data Performance Metrics:")
print(f"\nMAE: {mae}, RMSE: {rmse}")


Processing for country: FRA

LSTM Model Trained!


Testing (Out-of-Sample) Data Performance Metrics:

MAE: 170.20174883355037, RMSE: 200.414019215614


## Global Explainability Analysis of features (w.r.t. Pearson Correlation coefficients)  for "FRA" (France)

In [89]:
train_yhat = model.predict(train_X_scaled)# Concatenate the training and testing predictions and actual values
all_predictions = np.concatenate((train_yhat.ravel(), yhat.ravel()))
all_actual = np.concatenate((train_y, test_y))

# Now, create a DataFrame that aligns the predictions with the true values
predictions_df = pd.DataFrame({
    'Predicted': all_predictions,
    'Actual': all_actual
})

# Calculate correlations for each feature in the original dataset against the actual target
correlations = {}
for feature in feature_columns:
    # Extract the feature data from the original DataFrame, making sure to align it with the concatenated predictions
    # This assumes that the feature data is properly aligned with the target variable
    feature_data_aligned = iso_covid_df[feature][-len(all_predictions):].values
    corr, _ = pearsonr(feature_data_aligned, predictions_df['Actual'])
    correlations[feature] = corr

# Sort features by their correlation strength
sorted_features = sorted(correlations.items(), key=lambda x: abs(x[1]), reverse=True)

# Output the sorted features and their correlations
print(f"\nFeature Importance in forecasting new case count (based on Pearson Correlation) for country: {iso_code}:\n")
for feature, corr in sorted_features:
    print(f"{feature}: {corr}")


Feature Importance in forecasting new case count (based on Pearson Correlation) for country: FRA:

hosp_patients_per_million: 0.4465054530786238
tests_per_case: -0.30737372677956054
reproduction_rate: -0.27356392253410616
new_vaccinations_smoothed_per_million: -0.16399819186780573
icu_patients_per_million: 0.11392776575723058
stringency_index: -0.061878767599066284
new_people_vaccinated_smoothed_per_hundred: -0.01930266190956477


## Local / Phased Explainability Analysis of features (w.r.t. SHAP values) for "FRA" (France)

In [94]:
# Use the training data for deep explainer => can use fewer instances
explainer = shap.DeepExplainer(model, train_X_scaled)
# explain the the testing instances (can use fewer instanaces)
# explaining each prediction requires 2 * background dataset size runs
shap_values = explainer.shap_values(train_X_scaled[715:718])
# init the JS visualization code
shap.initjs()
print("SHAP Contribution to Prediction at date 2022-01-10 - period of fastest growth for FRA (Omicron variant onset):\n")
shap.force_plot(explainer.expected_value[0], shap_values[0][2], train_X_scaled[717], feature_names = feature_columns, out_names=forecast_column)

SHAP Contribution to Prediction at date 2022-01-10 - period of fastest growth for FRA (Omicron variant onset):



## Training and Testing LSTM Model for "ITA" (Italy)

In [96]:
iso_code = "ITA"
print(f"\nProcessing for country: {iso_code}")

# Data Preparation
iso_covid_df = prepare_data(covid_df, iso_code)

# Split and Scale Data
train_X_scaled, train_y, test_X_scaled, test_y, scaler = split_and_scale_data_lstm(iso_covid_df, train_ratio=0.8)

# Fit LSTM Model
# Define LSTM model
model = createLSTMModel((train_X_scaled.shape[1], train_X_scaled.shape[2]), l1Nodes=128, d1Nodes=64, d2Nodes=32)

# Fit model
history = model.fit(train_X_scaled, train_y, epochs=100, batch_size=8, validation_data=(test_X_scaled, test_y), verbose=0, shuffle=False)
print("\nLSTM Model Trained!\n")
yhat = model.predict(test_X_scaled)

# Evaluate the model
mae, rmse = evaluate_forecast(test_y, yhat)
print("\nTesting (Out-of-Sample) Data Performance Metrics:")
print(f"\nMAE: {mae}, RMSE: {rmse}")


Processing for country: ITA

LSTM Model Trained!


Testing (Out-of-Sample) Data Performance Metrics:

MAE: 103.34495596516929, RMSE: 126.03914863093623


## Global Explainability Analysis of features (w.r.t. Pearson Correlation coefficients)  for "ITA" (Italy)

In [98]:
train_yhat = model.predict(train_X_scaled)# Concatenate the training and testing predictions and actual values
all_predictions = np.concatenate((train_yhat.ravel(), yhat.ravel()))
all_actual = np.concatenate((train_y, test_y))

# Now, create a DataFrame that aligns the predictions with the true values
predictions_df = pd.DataFrame({
    'Predicted': all_predictions,
    'Actual': all_actual
})

# Calculate correlations for each feature in the original dataset against the actual target
correlations = {}
for feature in feature_columns:
    # Extract the feature data from the original DataFrame, making sure to align it with the concatenated predictions
    # This assumes that the feature data is properly aligned with the target variable
    feature_data_aligned = iso_covid_df[feature][-len(all_predictions):].values
    corr, _ = pearsonr(feature_data_aligned, predictions_df['Actual'])
    correlations[feature] = corr

# Sort features by their correlation strength
sorted_features = sorted(correlations.items(), key=lambda x: abs(x[1]), reverse=True)

# Output the sorted features and their correlations
print(f"\nFeature Importance in forecasting new case count (based on Pearson Correlation) for country: {iso_code}:\n")
for feature, corr in sorted_features:
    print(f"{feature}: {corr}")


Feature Importance in forecasting new case count (based on Pearson Correlation) for country: ITA:

tests_per_case: -0.38553470602979933
new_vaccinations_smoothed_per_million: -0.2850528979262922
reproduction_rate: -0.266659236821818
hosp_patients_per_million: 0.17937203653247744
stringency_index: -0.04486050511274169
new_people_vaccinated_smoothed_per_hundred: -0.027531239804383614
icu_patients_per_million: -0.007949465766966361


## Local / Phased Explainability Analysis of features (w.r.t. SHAP values) for "ITA" (Italy)

In [99]:
# Use the training data for deep explainer => can use fewer instances
explainer = shap.DeepExplainer(model, train_X_scaled)
# explain the the testing instances (can use fewer instanaces)
# explaining each prediction requires 2 * background dataset size runs
shap_values = explainer.shap_values(train_X_scaled[715:718])
# init the JS visualization code
shap.initjs()
print("SHAP Contribution to Prediction at date 2022-01-10 - period of fastest growth for ITA (Omicron variant onset):\n")
shap.force_plot(explainer.expected_value[0], shap_values[0][2], train_X_scaled[717], feature_names = feature_columns, out_names=forecast_column)

SHAP Contribution to Prediction at date 2022-01-10 - period of fastest growth for ITA (Omicron variant onset):

